In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import os
import pickle
import pandas as pd
import glob
import joblib as jl
import json

In [16]:
rootdir = "/nfs/nhome/live/npatel/projects/Collective-learning-/plotting/collective_learning/test_ode/2025-09-14_09-22-18/T2lr1.0r_11.0r_21.0r_120.0tau_10.0tau_20.0J_10.073J_2-0.0238Q_10.989Q_21.02Q_12-0.00716S0.967epochs100savepoints100"
df=pd.read_pickle(os.path.join(rootdir, 'ode_results.pkl'))
configs = json.load(open(os.path.join(rootdir, 'config.json')))

epochs = configs['fixed']['epochs']
savepoints = configs['fixed']['savepoints']
steps = np.unique(np.logspace(0, np.log10(epochs - 1), num=savepoints, dtype=int))
true_savepoints = np.shape(steps)[0]

configs['fixed'].pop('epochs', None)
configs['fixed'].pop('savepoints', None)
configs['fixed'].pop('logdir', None)
configs['fixed'].pop('epochs', None)
configs['fixed'].pop('D', None)

1000

In [17]:


if 'r_1s' in configs['sweep']:
    r_1_s = np.array(configs['sweep']['r_1s'])
else:
    r_1 = np.array(configs['fixed']['r_1s'])

if 'r_2s' in configs['sweep']:
    r_2_s = np.array(configs['sweep']['r_2s'])
else:
    r_2 = np.array(configs['fixed']['r_2s'])

if 'r_12s' in configs['sweep']:
    r_12_s = np.array(configs['sweep']['r_12s'])
else:
    r_12 = np.array(configs['fixed']['r_12s'])

if 'tau_1s' in configs['sweep']:
    tau_1_s = np.array(configs['sweep']['tau_1s'])
else:
    tau_1 = np.array(configs['fixed']['tau_1s'])

if 'tau_2s' in configs['sweep']:
    tau_2_s = np.array(configs['sweep']['tau_2s'])
else:
    tau_2 = np.array(configs['fixed']['tau_2s'])


In [18]:
"""
set the fixed parameters
"""
fixed_param_names = list(configs['fixed'].keys())
fixed_params = {}
for key in fixed_param_names:
    fixed_params[key[:-1]] = configs['fixed'][key][0]
print(fixed_params)
df_filtered = df.xs(tuple(fixed_params.values()), level=list(fixed_params.keys()))

{'lr': 1.0, 'r_1': 1.0, 'r_2': 1.0, 'tau_1': 0.0, 'tau_2': 0.0, 'J_1_init': 0.07301539468923571, 'J_2_init': -0.02381823273429834, 'Q_1_init': 0.988561558575577, 'Q_2_init': 1.0177215858998647, 'Q_12_init': -0.007158579290475447, 'S': 0.9672523256140616}


In [24]:
configs['fixed']['Ss'][0]

0.9672523256140616

In [20]:
"""
print swept parameters and values
"""
sweep_param_names = list(configs['sweep'].keys())
print({sweep_param_names[0]},  configs['sweep'][sweep_param_names[0]])
print({sweep_param_names[1]},  configs['sweep'][sweep_param_names[1]])

{'Ts'} [2, 3, 4]
{'r_12s'} [0.0, 10.0, 20.0]


In [ ]:
"""
The following is for a brief preliminary visisualization of results, and is absolutely not necessary.
The gist is to plot sweeps of trajectories for one of the parameters, while holding the other parameter fixed.
But this is done for a few values of the second parameter, with an individual subplot for each value of the second parameter.

The input would be be two lists of values for the two parameters being swept over
(this would typically be the full list of values for one parameter, 
and a smaller list of values for the second parameter. N.B. the smaller list should be a subset of the full list)

given df_filtered (filtered dataframe by fixed parameters),
sweep_params (a list of values of the parameters being swept and plotted over)
second_params (a (smaller) list of values of the secondary parameters being swept over and plotted in an outer loop)
sweep_param_name (name of the parameter being swept over)
second_param_name (name of the secondary parameter being 'swept' over)
"""
def plot_variable(df_filtered, variable, sweep_params, second_params, sweep_param_name, second_param_name, S, log_epochs):
    sweep_length = len(sweep_params)
    second_sweep_length = len(second_params)

    if 'rhos' in configs['sweep']:
        if 'rho' == sweep_param_name:
            rhos = np.array(sweep_params)
            Sr = S_r(rhos)
            Sr = np.tile(Sr, (second_sweep_length, 1))
            Si = S_i(rhos)
            Si = np.tile(Si, (second_sweep_length, 1))

        else:
            rhos = np.array(second_params)
            Sr = S_r(rhos)
            Sr = np.tile(Sr, (1, sweep_length))
            Si = S_i(rhos)
            Si = np.tile(Si, (1, sweep_length))
     
    else:
        rho = np.array(configs['fixed']['rhos'] * sweep_length)
        Sr = S_r(rho)
        Sr = np.tile(Sr, (second_sweep_length, 1))
        Si = S_i(rho)
        Si = np.tile(Si, (second_sweep_length, 1))

    if 'deltas' in configs['sweep']:
        if 'delta' == sweep_param_name:
            deltas = np.array(sweep_params)
            deltas = np.tile(deltas, (second_sweep_length, 1))
        else:
            deltas = np.array(second_params)
            deltas = np.tile(deltas, (1, sweep_length))
        deltas = np.array(configs['sweep']['deltas'])
    else:
        deltas = configs['fixed']['deltas'] * sweep_length

    if 'epsilons' in configs['sweep']:
        epsilons = np.array(configs['sweep']['epsilons'])
    else:
        epsilons = configs['fixed']['epsilons'] * sweep_length

    fig, axes = plt.subplots(len(second_params), 3, figsize=(10, 3*len(second_params)), sharex=True)
    for i, second_param in enumerate(second_params):
        df_filtered_new = df_filtered.xs(second_param, level=second_param_name)
        for param, row in df_filtered_new.iterrows():
            J_1 = row['J_1']
            J_2 = row['J_2']
            Q_1 = row['Q_1']
            Q_2 = row['Q_2']
            Q_12 = row['Q_12']

            axes[i,0].plot(log_epochs, J_1/np.sqrt(Q_1*S), label=f"{second_param_name}={second_param:.3g}")
            axes[i,1].plot(log_epochs, J_2/np.sqrt(Q_2*S), label=f"{second_param_name}={second_param:.3g}")
            axes[i,2].plot(log_epochs, Q_12/np.sqrt(Q_1*Q_2), label=f"{second_param_name}={second_param:.3g}")

            axes[i,0].set_title(f"rho_1 vs time (sweep over {sweep_param_name})")
            axes[i,1].set_title(f"rho_2 vs time (sweep over {sweep_param_name})")
            axes[i,2].set_title(f"12_overlap vs time (sweep over {sweep_param_name})")

            """axes[i,0].legend()
            axes[i,1].legend()
            axes[i,2].legend()"""

            axes[i,0].set_ylabel('rho_1')
            axes[i,1].set_ylabel('rho_2')
            axes[i,2].set_ylabel('12_overlap')
            axes[i,0].grid(True)
            axes[i,1].grid(True)
            axes[i,2].grid(True)
            axes[i,0].set_xscale('log')
            axes[i,1].set_xscale('log')
            axes[i,2].set_xscale('log')


        for param in sweep_params:
            try:
                y_vals = df_filtered.loc[(rho, delta)][variable]
                plt.plot(range(len(y_vals)), y_vals, label=f"rho={rho}, delta={delta}")
            except KeyError:
                print(f"Skipping rho={rho}, delta={delta} (not found in data)")
                continue
    plt.title(f"{variable} over time for varying rho and delta")
    plt.xlabel("Time Step")
    plt.ylabel(variable)
    plt.legend()
    plt.grid(True)
    plt.show()

# Example: Plot R, Qr, and Qi
plot_variable(df_filtered, 'R', rho_values_to_plot, delta_values_to_plot)
plot_variable(df_filtered, 'Qr', rho_values_to_plot, delta_values_to_plot)
plot_variable(df_filtered, 'Qi', rho_values_to_plot, delta_values_to_plot)
